In [ ]:
!pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53895 sha256=e9c1dddd973d7d791a6e4a064190f7d5c4b6b2580ae0abe212669a0d14cdfbc0
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [ ]:
!pip install textract

In [3]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.0 MB/s eta 0:00:00


In [8]:
import random
import time
import docx






def read_text(file_path):
    if file_path.endswith('.docx'):
        return read_docx(file_path)
    elif file_path.endswith('.txt'):
        return read_txt(file_path)
    else:
        raise ValueError("Unsupported file format")

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)


def read_txt(file_path):
    with open(file_path, 'r') as f:
        return f.read()


def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

def binary_to_text(binary_str):
    return ''.join(chr(int(binary_str[i:i+8], 2)) for i in range(0, len(binary_str), 8))

def divide_into_blocks(bit_stream):
    blocks = []
    while bit_stream:
        block_size = 8  # Fixed block size to 8 bits
        if len(bit_stream) < block_size:
            block_size = len(bit_stream)
        blocks.append(bit_stream[:block_size])
        bit_stream = bit_stream[block_size:]
    return blocks

def triangular_encrypt_block(bit_stream):
    triangle = [list(bit_stream)]
    triangle_output = [f"Iteration-1 {' '.join(list(bit_stream))}"]

    for iteration_index in range(len(bit_stream) - 1):
        new_iteration = []
        for j in range(len(triangle[iteration_index]) - 1):
            new_bit = str(int(triangle[iteration_index][j]) ^ int(triangle[iteration_index][j + 1]))
            new_iteration.append(new_bit)
        triangle.append(new_iteration)
        triangle_output.append(f"Iteration-{iteration_index + 2} {' '.join(new_iteration)}")

    option = random.choice(['001', '010', '011', '100'])
    arrow = ""
    if option == '001':
        encrypted_stream = ''.join(row[0] for row in triangle)
        arrow = "↓ (MSB source to last)"
    elif option == '010':
        encrypted_stream = ''.join(row[0] for row in reversed(triangle))
        arrow = "↑ (MSB last to source)"
    elif option == '011':
        encrypted_stream = ''.join(row[-1] for row in triangle)
        arrow = "↓ (LSB source to last)"
    else:  # '100'
        encrypted_stream = ''.join(row[-1] for row in reversed(triangle))
        arrow = "↑ (LSB last to source)"

    # Add the arrow to each iteration string based on the chosen option
    for i in range(len(triangle_output)):
        if ((option == '001' or option == '011') and i == 0) or \
           ((option == '010' or option == '100') and i == len(triangle_output) - 1):
            triangle_output[i] += " " + arrow

    return encrypted_stream, triangle_output, option


def encrypt_file(input_file, output_file, target_block_file, options_file, input_block_file):
    text = read_text(input_file)  # Modified to read text from any supported file type
    binary_text = text_to_binary(text)
    formatted_binary_text = '/'.join([binary_text[i:i+8] for i in range(0, len(binary_text), 8)])
    blocks = divide_into_blocks(binary_text)

    with open(output_file, 'w') as f, open(target_block_file, 'w') as tb, open(options_file, 'w') as op, open(input_block_file, 'w') as ib:
        f.write(f"Binary Representation of Input Text: {formatted_binary_text}\n\n")

        for block in blocks:
            encrypted_stream, triangle_output, option = triangular_encrypt_block(block)
            op.write(f"{option}\n")
            ib.write(f"{block}\n")  # Write each 8-bit block to InputBlock.txt
            ascii_char = binary_to_text(encrypted_stream)
            ascii_numeric_value = int(encrypted_stream, 2)
            f.write(f"{len(block)}-bit Input Block: {block} | Option: {option}\n")
            f.write("Triangle Structure:\n")
            for iteration in triangle_output:
                f.write(f"{iteration}\n")
            f.write("\n")
            tb.write(f"Target Block: {encrypted_stream} | ASCII Numeric Value: {ascii_numeric_value} | Cypher Text: {ascii_char}\n")


def extract_ascii_and_convert(target_block_file, encrypted_file):
    try:
        with open(target_block_file, 'r') as tb:
            target_blocks = tb.readlines()

        concatenated_target_blocks = ''
        for line in target_blocks:
            parts = line.split('|')
            if len(parts) > 1:
                binary_block = parts[0].strip().split()[-1]
                concatenated_target_blocks += binary_block

        with open(encrypted_file, 'w') as ef:
            ef.write(f"Concatenated Target Blocks: {concatenated_target_blocks}\n")
            cipher_text = binary_to_text(concatenated_target_blocks)
            ef.write(f"Cipher Text: {cipher_text}\n")

        print("Extraction and conversion completed successfully.")
    except FileNotFoundError:
        print("Target block file not found.")

# File paths (Update these paths as per your environment)
input_file = '/content/drive/MyDrive/inputfile.docx'
output_file = '/content/drive/MyDrive/output.txt'
target_block_file_path = '/content/drive/MyDrive/TargetBlock.txt'
input_block_file_path = '/content/drive/MyDrive/InputBlock.txt'
options_file_path = '/content/drive/MyDrive/options.txt'
encrypt_file_path = '/content/drive/MyDrive/encrypt.txt'
extract_ascii_and_convert_path = '/content/drive/MyDrive/cypher.txt'

# Execute the encryption process to generate target blocks and store options
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)

# Execute the encryption process to generate target blocks
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)


# Then call the functions as before
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)








start_time = time.time()

# Execute the encryption process to generate target blocks and store options
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)

# Execute the encryption process to generate target blocks
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)

# Record end time
end_time = time.time()

# Calculate total time taken in seconds
total_time = end_time - start_time

print(f"Overall Encryption time taken: {total_time:.2f} seconds")

Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Overall Encryption time taken: 0.11 seconds


In [12]:
from google.colab import files
import random
import time
import docx

def read_text(file_path):
    if file_path.endswith('.docx'):
        return read_docx(file_path)
    elif file_path.endswith('.txt'):
        return read_txt(file_path)
    else:
        raise ValueError("Unsupported file format")

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)

def read_txt(file_path):
    with open(file_path, 'r') as f:
        return f.read()

def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

def binary_to_text(binary_str):
    return ''.join(chr(int(binary_str[i:i+8], 2)) for i in range(0, len(binary_str), 8))

def divide_into_blocks(bit_stream):
    blocks = []
    while bit_stream:
        block_size = 8  # Fixed block size to 8 bits
        if len(bit_stream) < block_size:
            block_size = len(bit_stream)
        blocks.append(bit_stream[:block_size])
        bit_stream = bit_stream[block_size:]
    return blocks

def triangular_encrypt_block(bit_stream):
    triangle = [list(bit_stream)]
    triangle_output = [f"Iteration-1 {' '.join(list(bit_stream))}"]

    for iteration_index in range(len(bit_stream) - 1):
        new_iteration = []
        for j in range(len(triangle[iteration_index]) - 1):
            new_bit = str(int(triangle[iteration_index][j]) ^ int(triangle[iteration_index][j + 1]))
            new_iteration.append(new_bit)
        triangle.append(new_iteration)
        triangle_output.append(f"Iteration-{iteration_index + 2} {' '.join(new_iteration)}")

    option = random.choice(['001', '010', '011', '100'])
    arrow = ""
    if option == '001':
        encrypted_stream = ''.join(row[0] for row in triangle)
        arrow = "↓ (MSB source to last)"
    elif option == '010':
        encrypted_stream = ''.join(row[0] for row in reversed(triangle))
        arrow = "↑ (MSB last to source)"
    elif option == '011':
        encrypted_stream = ''.join(row[-1] for row in triangle)
        arrow = "↓ (LSB source to last)"
    else:  # '100'
        encrypted_stream = ''.join(row[-1] for row in reversed(triangle))
        arrow = "↑ (LSB last to source)"

    # Add the arrow to each iteration string based on the chosen option
    for i in range(len(triangle_output)):
        if ((option == '001' or option == '011') and i == 0) or \
           ((option == '010' or option == '100') and i == len(triangle_output) - 1):
            triangle_output[i] += " " + arrow

    return encrypted_stream, triangle_output, option

def encrypt_file(input_file, output_file, target_block_file, options_file, input_block_file):
    text = read_text(input_file)  # Modified to read text from any supported file type
    binary_text = text_to_binary(text)
    formatted_binary_text = '/'.join([binary_text[i:i+8] for i in range(0, len(binary_text), 8)])
    blocks = divide_into_blocks(binary_text)

    with open(output_file, 'w') as f, open(target_block_file, 'w') as tb, open(options_file, 'w') as op, open(input_block_file, 'w') as ib:
        f.write(f"Binary Representation of Input Text: {formatted_binary_text}\n\n")

        for block in blocks:
            encrypted_stream, triangle_output, option = triangular_encrypt_block(block)
            op.write(f"{option}\n")
            ib.write(f"{block}\n")  # Write each 8-bit block to InputBlock.txt
            ascii_char = binary_to_text(encrypted_stream)
            ascii_numeric_value = int(encrypted_stream, 2)
            f.write(f"{len(block)}-bit Input Block: {block} | Option: {option}\n")
            f.write("Triangle Structure:\n")
            for iteration in triangle_output:
                f.write(f"{iteration}\n")
            f.write("\n")
            tb.write(f"Target Block: {encrypted_stream} | ASCII Numeric Value: {ascii_numeric_value} | Cypher Text: {ascii_char}\n")

def extract_ascii_and_convert(target_block_file, encrypted_file):
    try:
        with open(target_block_file, 'r') as tb:
            target_blocks = tb.readlines()

        concatenated_target_blocks = ''
        for line in target_blocks:
            parts = line.split('|')
            if len(parts) > 1:
                binary_block = parts[0].strip().split()[-1]
                concatenated_target_blocks += binary_block

        with open(encrypted_file, 'w') as ef:
            ef.write(f"Concatenated Target Blocks: {concatenated_target_blocks}\n")
            cipher_text = binary_to_text(concatenated_target_blocks)
            ef.write(f"Cipher Text: {cipher_text}\n")

        print("Extraction and conversion completed successfully.")
    except FileNotFoundError:
        print("Target block file not found.")

# Upload files
uploaded = files.upload()

# Choose the file to encrypt
input_file = list(uploaded.keys())[0]  # Assuming only one file is uploaded

# File paths (Update these paths as per your environment)
output_file = '/content/drive/MyDrive/output.txt'
target_block_file_path = '/content/drive/MyDrive/TargetBlock.txt'
input_block_file_path = '/content/drive/MyDrive/InputBlock.txt'
options_file_path = '/content/drive/MyDrive/options.txt'
encrypt_file_path = '/content/drive/MyDrive/encrypt.txt'
extract_ascii_and_convert_path = '/content/drive/MyDrive/cypher.txt'

# Execute the encryption process to generate target blocks and store options
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)


# Execute the encryption process to generate target blocks and store options
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)

# Execute the encryption process to generate target blocks
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)


# Then call the functions as before
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)






start_time = time.time()

# Execute the encryption process to generate target blocks and store options
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)

# Execute the encryption process to generate target blocks
encrypt_file(input_file, output_file, target_block_file_path, options_file_path, input_block_file_path)

# Extract and convert the concatenated target blocks into cipher text
extract_ascii_and_convert(target_block_file_path, extract_ascii_and_convert_path)

# Record end time
end_time = time.time()

# Calculate total time taken in seconds
total_time = end_time - start_time

print(f"Overall Encryption time taken: {total_time:.2f} seconds")


Saving input.txt to input (2).txt
Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Extraction and conversion completed successfully.
Overall Encryption time taken: 0.10 seconds


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
